In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Swiggy_FAQ.csv",encoding_errors="ignore")

In [3]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation
from texthero.preprocessing import remove_digits

In [4]:
from sentence_transformers import SentenceTransformer
from torch.nn import CosineSimilarity
import torch
import pickle

In [5]:
pd.set_option('display.max_colwidth', None)
df

,Q,A
0,Does Swiggy offer massages?,Good perks and benefits like free transport and team outings in 3 mnths
1,Does Swiggy offer dental insurance?,Benefits (nothing outstanding apart from a standard medical insurance )
2,Does Swiggy offer employee discounts?,No employee discount in app
3,Does Swiggy offer life insurance?,Benefits programs must be reviewed
4,Does Swiggy assist with or provide childcare?,Good benefits for employees .
5,What is health insurance like at Swiggy?,"The company cares for each employee and they provide certain benefits for them too be it health benefits, Esic etc"
6,Does Swiggy offer sponsored degrees?,5 days working & fixed 5k reimbursement.
7,Does Swiggy offer unlimited holiday?,"Good Environment, helpful colleagues, time flexibility, weekends off"
8,Does Swiggy offer vision insurance?,"Free food, time, leaves, pickups"
9,Does Swiggy offer disability insurance?,"Additionally, it seems the senior people are more focussed on short term benefits only even at the cost of mid to long term gains."


In [6]:
df['procd_Q'] = df['Q'].pipe(remove_digits).pipe(remove_punctuation).pipe(remove_lessthan,length=3)
df

C:\Users\SREENEHA\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,Q,A,procd_Q
0,Does Swiggy offer massages?,Good perks and benefits like free transport and team outings in 3 mnths,Does Swiggy offer massages
1,Does Swiggy offer dental insurance?,Benefits (nothing outstanding apart from a standard medical insurance ),Does Swiggy offer dental insurance
2,Does Swiggy offer employee discounts?,No employee discount in app,Does Swiggy offer employee discounts
3,Does Swiggy offer life insurance?,Benefits programs must be reviewed,Does Swiggy offer life insurance
4,Does Swiggy assist with or provide childcare?,Good benefits for employees .,Does Swiggy assist with provide childcare
5,What is health insurance like at Swiggy?,"The company cares for each employee and they provide certain benefits for them too be it health benefits, Esic etc",What health insurance like Swiggy
6,Does Swiggy offer sponsored degrees?,5 days working & fixed 5k reimbursement.,Does Swiggy offer sponsored degrees
7,Does Swiggy offer unlimited holiday?,"Good Environment, helpful colleagues, time flexibility, weekends off",Does Swiggy offer unlimited holiday
8,Does Swiggy offer vision insurance?,"Free food, time, leaves, pickups",Does Swiggy offer vision insurance
9,Does Swiggy offer disability insurance?,"Additionally, it seems the senior people are more focussed on short term benefits only even at the cost of mid to long term gains.",Does Swiggy offer disability insurance


In [7]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [8]:
MODEL_PATH = r"model_va.pickle"
q_embs = model.encode(df["procd_Q"]) # computes encode for all the questions from the dataset. 
                                    #Embeddings can be computed in batches for massive dataset.
with open(MODEL_PATH, "wb") as file:
    pickle.dump(q_embs, file)

In [9]:
with open(r"model_va.pickle", "rb") as file:
    q_embs = pickle.load(file)

In [10]:
def pred_answer(usr_query):
    df_query = pd.DataFrame([usr_query], columns=["usr_query"]) # use similar pipeline that was used for computing embeddings from dataset
    df_query["clean_usr_q"] = df_query["usr_query"].pipe(remove_digits).pipe(remove_punctuation)
    usr_q_emb = model.encode(df_query["clean_usr_q"]) # compute embedding
    cosine_similarity = CosineSimilarity()
    q_idx = np.argmax(cosine_similarity(torch.from_numpy(usr_q_emb), torch.from_numpy(q_embs))) # compute cosine similarity and find the matched query
    return (df["A"][q_idx.item()]) # look up answer of the matched query from the dataframe of input dataset

In [11]:
while True:
    usr_query = input("Ask a query:")
    if usr_query == "exit":
        break
    else:
        print("Answer: ", pred_answer(usr_query))
    print("-----------------")

Ask a query:what is the salary at swiggy?


C:\Users\SREENEHA\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


Answer:  No employee discount in app
-----------------
Ask a query:what is the insurance?
Answer:  The company cares for each employee and they provide certain benefits for them too be it health benefits, Esic etc
-----------------
Ask a query:Is there promotions?
Answer:  Growth and more opportunities for promotions I found here.
-----------------
Ask a query:exit
